## Пример построения распределения температуры с помощью модуля uTemp.py

In [2]:
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import scipy as sp
import sys
import pylab
from scipy.optimize import fsolve


from bokeh.layouts import column,row,layout
from bokeh.models import ColumnDataSource, Slider, RadioButtonGroup,TableColumn, DataTable, Range1d, LinearAxis
from bokeh.plotting import figure, output_file, show
from bokeh.themes import Theme
from bokeh.io import show, output_notebook, output_file, push_notebook
from bokeh.layouts import widgetbox
from bokeh.models.tools import HoverTool, PointDrawTool

sys.path.append('../')
import uPVT.PVT as PVT
import uSep.separation as sp
import uTemperature.uTemp as temp

output_notebook()

Loading BokehJS ...

### Задание необходимых свойств через класс `system_properties`
При создании экземляра класса `sys_prop` в нем уже есть все необходимые для расчета параметры, которые при необходимости можно менять

In [4]:
sys_prop=temp.system_properties()

###  Пример построения КРТ и КРД
В данном варианте для простоты строим линейное распределение давления

In [6]:
depth_m = 2550
pbh_pa=25*10**6
c=pbh_pa/depth_m
tei=sys_prop.tei_c
gg=sys_prop.gg_cm
tff, tfg, p=[], [], []

for i in range(1,depth_m,10):
    p_val=pbh_pa-c*i
    tg=tei-gg*i
    tfg.append(tg)
    p.append(p_val)
l=np.asarray(list(range(1,depth_m,10)))
p=np.asarray(p)
tfg=np.asarray(tfg)
mistake=p*0+1

tff=temp.temp_fluid(l,p,mistake,sys_prop)
def make():
    plot = figure(title='Распределение температуры',
                   x_axis_label='Tемпература, °C',
                   y_axis_label='Расстояние от забоя,м',
                )
    plot.line(tff,l,color='red',legend='Температура флюида')
    plot.line(tfg,l,legend='Геотерма')
    plot.line(p/10**5,l,legend='Давление, атм',color='black')
    show(plot) 
make()

### Влияние дебита на КРТ
Обратите внимание: через цикл меняются параметр в экземляре класса

In [23]:
sys_prop_q=temp.system_properties()
ql=np.asarray([10,30,50,100,150])/86400

plot = figure(title='Распределение температуры',
               x_axis_label='Tемпература, °C',
               y_axis_label='Глубина от забоя,м')
kk=['red','green','blue','yellow','black']
for i,k in zip(ql,kk):
    sys_prop_q.ql_m3sec=i
    tff=temp.temp_fluid(l,p,mistake,sys_prop_q)
    legendd='Q='+str(i*86400) + 'м3 /сут'
    plot.line(tff,l,color=k,legend=legendd)
show(plot)

### Влияние ГФ на КРТ

In [7]:
start = sys_prop.qg_m3sec
sys_prop_game=temp.system_properties()
qg=np.asarray([start*1,start*2,start*5,start*10,start*20])
plot = figure(title='Распределение температуры',
               x_axis_label='Tемпература, °C',
               y_axis_label='Глубина от забоя,м')
kk=['red','green','blue','yellow','black']
for i,k in zip(qg,kk):
    sys_prop_game.qg_m3sec=i
    tff=temp.temp_fluid(l,p,mistake,sys_prop_game)
    legendd='ГФ='+str(i/sys_prop_game.ql_m3sec) + 'м3 /сут'
    plot.line(tff,l,color=k,legend=legendd)
show(plot)